In [ ]:
# Install SimpleITK
!pip install SimpleITK

#Import modules
import numpy as np
import nibabel as nib
from sklearn.mixture import GaussianMixture

import os
import sys

%matplotlib inline
import matplotlib.pyplot as plt

import SimpleITK as sitk
import nibabel as nib
import pandas as pd
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import precision_score, recall_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 15.0 MB/s eta 0:00:00


In [ ]:
# mount drive with files

from google.colab import drive
drive.mount('/content/drive')


!pip install -q "monai-weekly[nibabel, tqdm, einops]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
#Importing all epoch losses and metric values from each augmentations + UNet

unet_no_aug_epoch_loss = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_epoch_loss_values_unetwithaug_ABSOLUTELY_NOAUG.npy')
unet_no_aug_metric_values = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_metric_values_unetwithaug_ABSOLUTELY_NOAUG.npy')

unet_cropforegroundd_epoch_loss = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_epoch_loss_values_unetwithaug_CropForegroundd.npy')
unet_cropforegroundd_metric_values = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_metric_values_unetwithaug_CropForegroundd.npy')

unet_RandAdjustContrastd_epoch_loss = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_epoch_loss_values_unetwithaug_RandAdjustContrastd.npy')
unet_RandAdjustContrastd_metric_values = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_metric_values_unetwithaug_RandAdjustContrastd.npy')

unet_RandCoarseDropoutd_epoch_loss = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_epoch_loss_values_unetwithaug_RandCoarseDropoutd.npy')
unet_RandCoarseDropoutd_metric_values = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_metric_values_unetwithaug_RandCoarseDropoutd.npy')

unet_RandFlipd_epoch_loss = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_epoch_loss_values_unetwithaug_RandFlipd.npy')
unet_RandFlipd_metric_values = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_metric_values_unetwithaug_RandFlipd.npy')

unet_RandGaussianSharpend_epoch_loss = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_epoch_loss_values_unetwithaug_RandGaussianSharpend.npy')
unet_RandGaussianSharpend_metric_values = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_metric_values_unetwithaug_RandGaussianSharpend.npy')

unet_RandGaussianSmoothd_epoch_loss = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_epoch_loss_values_unetwithaug_RandGaussianSmoothd.npy')
unet_RandGaussianSmoothd_metric_values = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_metric_values_unetwithaug_RandGaussianSmoothd.npy')

unet_RandGibbsNoised_epoch_loss = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_epoch_loss_values_unetwithaug_RandGibbsNoised.npy')
unet_RandGibbsNoised_metric_values = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_metric_values_unetwithaug_RandGibbsNoised.npy')

unet_RandKSpaceSpikeNoised_epoch_loss = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_epoch_loss_values_unetwithaug_RandKSpaceSpikeNoised.npy')
unet_RandKSpaceSpikeNoised_metric_values = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_metric_values_unetwithaug_RandKSpaceSpikeNoised.npy')

unet_RandAffined_epoch_loss = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_epoch_loss_values_unetwithaug_RandAffined.npy')
unet_RandAffined_metric_values = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_metric_values_unetwithaug_RandAffined.npy')

unet_RandBiasFieldd_epoch_loss = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_epoch_loss_values_unetwithaug_RandBiasFieldd.npy')
unet_RandBiasFieldd_metric_values = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_metric_values_unetwithaug_RandBiasFieldd.npy')

In [ ]:
def Extract_x_y(loss_np_load):
  x = [(i + 1) for i in range(len(loss_np_load))]
  y = loss_np_load
  return x,y

In [ ]:
import plotly.graph_objects as go
import numpy as np


# Create traces
fig = go.Figure()
x, y = Extract_x_y(unet_no_aug_epoch_loss)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines+markers',
                    name='No Augmentations'))
x, y = Extract_x_y(unet_cropforegroundd_epoch_loss)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Crop Foreground'))
x, y = Extract_x_y(unet_RandAdjustContrastd_epoch_loss)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Adjust Contrast'))
x, y = Extract_x_y(unet_RandCoarseDropoutd_epoch_loss)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Coarse Dropout'))
x, y = Extract_x_y(unet_RandFlipd_epoch_loss)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Flipd'))
x, y = Extract_x_y(unet_RandGaussianSharpend_epoch_loss)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Gaussian Sharpen'))
x, y = Extract_x_y(unet_RandGaussianSmoothd_epoch_loss)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Gaussian Smooth'))
x, y = Extract_x_y(unet_RandGibbsNoised_epoch_loss)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Gibbs Noise'))
x, y = Extract_x_y(unet_RandKSpaceSpikeNoised_epoch_loss)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand KSpace Spike Noise'))
x, y = Extract_x_y(unet_RandAffined_epoch_loss)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Affine'))
x, y = Extract_x_y(unet_RandBiasFieldd_epoch_loss)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Bias Field'))





# Edit the layout
fig.update_layout(title='Iteration Average DiceCE Loss for Each Augmentation',
                   xaxis_title='Iteration',
                   yaxis_title='Average DiceCE Loss')


fig.show()

In [ ]:
import plotly.graph_objects as go
import numpy as np


# Create traces
fig = go.Figure()
x, y = Extract_x_y(unet_no_aug_metric_values)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines+markers',
                    name='No Augmentations'))

x, y = Extract_x_y(unet_cropforegroundd_metric_values)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Crop Foreground'))


x, y = Extract_x_y(unet_RandAdjustContrastd_metric_values)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Adjust Contrast'))

x, y = Extract_x_y(unet_RandCoarseDropoutd_metric_values)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Coarse Dropout'))

x, y = Extract_x_y(unet_RandFlipd_metric_values)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Flipd'))

x, y = Extract_x_y(unet_RandGaussianSharpend_metric_values)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Gaussian Sharpen'))

x, y = Extract_x_y(unet_RandGaussianSmoothd_metric_values)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Gaussian Smooth'))

x, y = Extract_x_y(unet_RandGibbsNoised_metric_values)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Gibbs Noise'))

x, y = Extract_x_y(unet_RandKSpaceSpikeNoised_metric_values)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand KSpace Spike Noise'))

x, y = Extract_x_y(unet_RandAffined_metric_values)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Affine'))

x, y = Extract_x_y(unet_RandBiasFieldd_metric_values)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='Rand Bias Field'))




# Edit the layout
fig.update_layout(title='Iteration Average Dice Score for Each Augmentation',
                   xaxis_title='Iteration',
                   yaxis_title='Average Dice Score')


fig.show()

In [ ]:
#EVALUATING NO AUGMENTATIONS VS ALL AUGMENTATIONS VS ONLY GOOD AUGMENTATIONS

unet_no_aug_epoch_loss = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_epoch_loss_values_unetwithaug_ABSOLUTELY_NOAUG.npy')
unet_no_aug_metric_values = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_metric_values_unetwithaug_ABSOLUTELY_NOAUG.npy')

unet_all_aug_epoch_loss = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_epoch_loss_values_unetwithaug_ALL_AUG.npy')
unet_all_aug_metric_values = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_metric_values_unetwithaug_ALL_AUG.npy')

unet_good_aug_epoch_loss = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_epoch_loss_values_unetwithaug_ONLY_GOOD_AUG.npy')
unet_good_aug_metric_values = np.load('/content/drive/MyDrive/face-seg-project-2023/new-training-062023/face_metric_values_unetwithaug_ONLY_GOOD_AUG.npy')





In [ ]:
# Create traces
fig = go.Figure()


x, y = Extract_x_y(unet_no_aug_epoch_loss)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='No Augmentations'))

x, y = Extract_x_y(unet_all_aug_epoch_loss)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='All Augmentations'))

x, y = Extract_x_y(unet_good_aug_epoch_loss)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='"Good" Augmentations*'))


# Edit the layout
fig.update_layout(title='Iteration Average DiceCE Loss for Each Augmentation Criteria',
                   xaxis_title='Iteration',
                   yaxis_title='Average DiceCE Loss')


fig.show()

In [ ]:
# Create traces
fig = go.Figure()


x, y = Extract_x_y(unet_no_aug_metric_values)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='No Augmentations'))

x, y = Extract_x_y(unet_all_aug_metric_values)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='All Augmentations'))

x, y = Extract_x_y(unet_good_aug_metric_values)
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='"Good" Augmentations*'))


# Edit the layout
fig.update_layout(title='Iteration Average Dice Score for Each Augmentation Criteria',
                   xaxis_title='Iteration',
                   yaxis_title='Average Dice Score')


fig.show()